In [ ]:
import numpy as np
import cv2
import glob
import yaml
import os


#import pathlab

corner_x=9 # number of chessboard corner in x direction
corner_y=6 # number of chessboard corner in x direction

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.01)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((corner_y*corner_x,3), np.float32)
objp[:,:2] = np.mgrid[0:corner_x,0:corner_y].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
jpegpoints = [] # 2d points in image plane.


source_path = "C:/Users/patir/Camera"
#print(os.getcwd())
print('image found :',len(os.listdir(source_path)))

images = [source_path + '/' + f for f in glob.glob('*.jpeg')]

#images = glob.glob('D:\Image Vision')


# path = 'results'
# pathlib.Path(path).mkdir(parents=True, exist_ok=True)

found = 0
for fname in images: # here, 10 can be changed to whatever number you like to choose
    jpeg = cv2.imread(fname) # capture frame by frame
    cv2.imshow('jpeg', jpeg)
    cv2.waitKey(500)
    print(fname)
    gray = cv2.cvtColor(jpeg, cv2.COLOR_BGRA2GRAY)
    
    # find the chess noard corners
    ret, corners = cv2.findChessboardCorners(gray, (corner_x,corner_y), None)
    # if found, ass object points, image points (after refining them)
    if ret == True:
        
        objpoints.append(objp) #Certainly, every loop objp is the same in 3D
        corners2 = cv2.cornerSubPix(gray,corners,(20,5),(-1,-5),criteria)
        jpegpoints.append(corners2)
        # Draw and display the corners
        jpeg = cv2.drawChessboardCorners(jpeg, (corner_x,corner_y), corners2, ret)
        found += 1
        cv2.imshow('chessboard', jpeg)
        cv2.waitKey(500)
        # if you want to save images with dtected corners
      
        
print("number of images used for calibration: ", found)
 # when everything done, release the capture
# cap.release()
# cv2.destroyAllWindows()

#calibration
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, jpegpoints, gray.shape[::-1], None, None)

# transforms the matrix distortion coefficients to writeable lists
data= {'camera_matrix': np.asarray(mtx).tolist(), 'dist_coeff': np.asarray(dist).tolist()}
print(mtx)
print(dist)
# and save it to a file
with open("calibration_matrix.yaml", "w")as f:
    yaml.dump(data, f)

#undistort image

for fname in images: # here, 10 can be changed to whatever number you like to choose
     #print(fname)
     jpeg = cv2.imread(fname) # Capture frame-by-frame
     #cv2.imshow('jpeg', jpeg)
     #cv2.waitkey(500)
     h, w = jpeg.shape[:2]
     
     newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx, dist , (w,h), 1, (w,h))
     #print(newcameramtx)
     #undistort
     dst = cv2.undistort(jpeg, mtx, dist, None, newcameramtx)
     cv2.imshow('undistort', dst)
     cv2.waitKey(500)
     
     # crop the image
     x, y, w, h = roi
     dst = dst[y:y+h, x:x+w]
     #cv2.imshow('calibration.png',dst)
     cv2.imshow('undistort2', dst)
     cv2.waitKey(0)
     
cv2.destroyAllWindows()


image found : 12
C:/Users/patir/Camera/WhatsApp Image 2022-06-24 at 00.40.19.jpeg
C:/Users/patir/Camera/WhatsApp Image 2022-06-24 at 00.40.20.jpeg
C:/Users/patir/Camera/WhatsApp Image 2022-06-24 at 00.40.21.jpeg
C:/Users/patir/Camera/WhatsApp Image 2022-06-24 at 00.40.22.jpeg
C:/Users/patir/Camera/WhatsApp Image 2022-06-24 at 00.40.25.jpeg
C:/Users/patir/Camera/WhatsApp Image 2022-06-24 at 00.40.27 (1).jpeg
C:/Users/patir/Camera/WhatsApp Image 2022-06-24 at 00.40.27.jpeg
C:/Users/patir/Camera/WhatsApp Image 2022-06-24 at 00.40.29.jpeg
C:/Users/patir/Camera/WhatsApp Image 2022-06-24 at 00.40.30.jpeg
number of images used for calibration:  9
[[1.56975484e+03 0.00000000e+00 7.98850457e+02]
 [0.00000000e+00 1.56699991e+03 6.37280372e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
[[-0.47661812  0.0753206   0.0018412   0.00109191  0.53154737]]


In [2]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(jpegpoints[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "total error: {}".format(mean_error/len(objpoints)) )

total error: 0.07649691877310993
